In [1]:
#import tflearn packages

import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression

#import other packages
import pandas as pd
import sklearn
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from skimage import measure
%matplotlib inline

In [2]:
#randomly choose a subset of training images to make it run faster

#import images

#label images by directory

#downsample images

In [3]:
#build a simple CNN

In [4]:
#test on subset of training images you didn't use; if it works well could do a full train:test and submit